# Ingest ISO 3166 data into Trino pipeline

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Run these in a notebook cell if you need to install onto your nb env

```python
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

In [ ]:
from dotenv import dotenv_values, load_dotenv
from osc_ingest_trino import *
import os
import pathlib

Load Environment Variables

In [ ]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [ ]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

In [ ]:
import osc_ingest_trino as osc

trino_bucket = attach_s3_bucket("S3_DEV")

Open a Trino connection using JWT for authentication

In [ ]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
essd_schema ='sandbox'

In [ ]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

The python module `pycountry` has all the data we need, and we can use it within python/pandas.

We import the data here so we can use it in SQL/Trino queries.

In [ ]:
import pycountry

Enter the Pandas!

In [ ]:
import pandas as pd
import numpy as np
import io

Drop previous tables and schema to start with a fresh slate

In [ ]:
for ingest_table in [ 'countries', 'subdivisions' ]:
    sql = f"""
drop table if exists {ingest_schema}.{ingest_table}
"""
    print(sql)
    qres = engine.execute(sql)
    print(qres.fetchall())

In [ ]:
sql = f"show tables in {ingest_schema}"
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())

One table we want to link up with is defined in the ESSD schema.  It links ISO Alpha3 codes with UN regions.

In [ ]:
engine.execute(f"select * from {essd_schema}.regions").fetchall()[0:5]

In [ ]:
df = pd.DataFrame([country.__dict__['_fields'] for country in pycountry.countries])
df.replace({np.NaN:pd.NaT}, inplace=True)
df_countries = df.convert_dtypes()
df_countries

In [ ]:
df = pd.DataFrame([subdivision.__dict__['_fields'] for subdivision in pycountry.subdivisions])
display(df[df.parent_code.notna()])
df.replace({np.NaN:pd.NaT}, inplace=True)
df_subdivisions = df.convert_dtypes()
df_subdivisions

In [ ]:
for ingest_table, df in [ ('countries',df_countries), ('subdivisions', df_subdivisions)]:
    table_check = engine.execute(f'drop table if exists {ingest_schema}.{ingest_table}')
    for row in table_check.fetchall():
        print(row)
    columnschema = create_table_schema_pairs(df)
    tabledef = f"""
create table if not exists {ingest_schema}.{ingest_table} (
{columnschema}
) with (
format = 'ORC'
)
"""
    print(tabledef)

    table_create = engine.execute(tabledef)
    for row in table_create.fetchall():
        print(row)
    df.to_sql(ingest_table,
              con=engine, schema=ingest_schema, if_exists='append',
              index=False,
              method=osc.TrinoBatchInsert(batch_size = 4000, verbose = False))
    dataset_query = (f'SELECT * FROM {ingest_schema}.{ingest_table} limit 10')
    print(dataset_query)
    dataset = engine.execute(dataset_query)
    for row in dataset.fetchall():
        print(row)